In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
from sklearn.preprocessing import label_binarize

# Загрузка данных
train_data = pd.read_csv('C:\\Users\\Karina\\Desktop\\Интенсив\\clean_train.csv')
valid_data = pd.read_csv('C:\\Users\\Karina\\Desktop\\Интенсив\\clean_valid.csv')

# Проверка данных
print("Train data columns:", train_data.columns)
print("Valid data columns:", valid_data.columns)

# Предположим, что целевая переменная называется 'Купит\некупит'
# и признаки - все остальные столбцы
try:
    X_train = train_data.drop(columns=['Купит\некупит'])  # Удаляем целевую переменную
    y_train = train_data['Купит\некупит']  # Целевая переменная

    X_valid = valid_data.drop(columns=['Купит\некупит'])  # Удаляем целевую переменную
    y_valid = valid_data['Купит\некупит']  # Целевая переменная
except KeyError as e:
    print(f"Ошибка: {e}. Убедитесь, что имя столбца 'Купит\некупит' указано верно.")
    raise

# Проверка на пропущенные значения
if X_train.isnull().any().any() or X_valid.isnull().any().any():
    print("В данных есть пропущенные значения. Удалите или заполните их.")
    raise ValueError("Пропущенные значения в данных.")

# Проверка соответствия признаков
if not set(X_train.columns) == set(X_valid.columns):
    print("Признаки в обучающих и тестовых данных не совпадают.")
    print("Признаки в X_train:", X_train.columns)
    print("Признаки в X_valid:", X_valid.columns)
    #raise ValueError("Несоответствие признаков.")

# Убедимся, что порядок столбцов совпадает
X_valid = X_valid[X_train.columns]

# Преобразование меток в бинарный формат для многоклассовой ROC-кривой
unique_classes = np.unique(y_train)
y_train_encoded = label_binarize(y_train, classes=unique_classes)
y_valid_encoded = label_binarize(y_valid, classes=unique_classes)

# Определение параметров модели для классификации (Random Forest)
rf_params = {
    'n_estimators': 100,
    'max_depth': 5,
    'random_state': 42
}

# Обучение модели RandomForestClassifier
rf_model = RandomForestClassifier(**rf_params)
rf_model.fit(X_train, y_train)

# Предсказание вероятностей для тестовых данных
y_pred_proba_rf = rf_model.predict_proba(X_valid)

# Построение кривой ROC для каждого класса
n_classes = y_train_encoded.shape[1]  # Количество классов
fpr_rf = dict()
tpr_rf = dict()
roc_auc_rf = dict()

# Проверка на бинарную классификацию
if n_classes == 1:
    n_classes = 2
    y_train_encoded = np.hstack((1 - y_train_encoded, y_train_encoded))
    y_valid_encoded = np.hstack((1 - y_valid_encoded, y_valid_encoded))

for i in range(n_classes):
    # Вычисление FPR и TPR для каждого класса
    fpr_rf[i], tpr_rf[i], _ = roc_curve(y_valid_encoded[:, i], y_pred_proba_rf[:, i])
    roc_auc_rf[i] = auc(fpr_rf[i], tpr_rf[i])

# Построение графика AUC-ROC
plt.figure(figsize=(10, 6))
for i in range(n_classes):
    plt.plot(fpr_rf[i], tpr_rf[i], label=f'ROC curve (class {i}, AUC = {roc_auc_rf[i]:.2f})')

# Добавление диагональной линии для сравнения
plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.5)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multi-Class Classification (Random Forest)')
plt.legend(loc="lower right")
plt.show()

# Преобразование вероятностей в предсказанные классы
y_pred_rf = np.argmax(y_pred_proba_rf, axis=1)

# Вывод отчета классификации для RandomForestClassifier
print("Random Forest Classification Report:")
print(classification_report(y_valid, y_pred_rf))

# Вывод матрицы ошибок для RandomForestClassifier
print("Random Forest Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_rf))